In [1]:
import train
import utils.plotter as plotter

data = train.create_dataset()
tte_test_data, tee_test_data = train.load_test_data()
for i in range(10):
    # plotter.plot_with_augmentations(tee_test_data, i)
    # plotter.plot_with_augmentations(tee_test_data, i)
    plotter.plot_with_augmentations(tee_test_data, i)


ValueError: Values for crop should be non negative and equal or smaller than image sizes(x_min = 0, y_min = 0, x_max = 302, y_max = 262, height = 1, width = 384)